# g

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec  6 03:15:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


#d

In [8]:
import os
import pickle
import matplotlib.pyplot as plt
import torch
import torch.optim as optim 
from torch import nn
import pandas as pd
import numpy as np
from torch.optim import lr_scheduler

In [5]:
test = pd.read_csv('drive/MyDrive/dacon/dataf/상추/train_input/CASE_01.csv')

In [ ]:
test.info()

In [13]:
print(dir(nn))

['AdaptiveAvgPool1d', 'AdaptiveAvgPool2d', 'AdaptiveAvgPool3d', 'AdaptiveLogSoftmaxWithLoss', 'AdaptiveMaxPool1d', 'AdaptiveMaxPool2d', 'AdaptiveMaxPool3d', 'AlphaDropout', 'AvgPool1d', 'AvgPool2d', 'AvgPool3d', 'BCELoss', 'BCEWithLogitsLoss', 'BatchNorm1d', 'BatchNorm2d', 'BatchNorm3d', 'Bilinear', 'CELU', 'CTCLoss', 'ChannelShuffle', 'ConstantPad1d', 'ConstantPad2d', 'ConstantPad3d', 'Container', 'Conv1d', 'Conv2d', 'Conv3d', 'ConvTranspose1d', 'ConvTranspose2d', 'ConvTranspose3d', 'CosineEmbeddingLoss', 'CosineSimilarity', 'CrossEntropyLoss', 'CrossMapLRN2d', 'DataParallel', 'Dropout', 'Dropout1d', 'Dropout2d', 'Dropout3d', 'ELU', 'Embedding', 'EmbeddingBag', 'FeatureAlphaDropout', 'Flatten', 'Fold', 'FractionalMaxPool2d', 'FractionalMaxPool3d', 'GELU', 'GLU', 'GRU', 'GRUCell', 'GaussianNLLLoss', 'GroupNorm', 'Hardshrink', 'Hardsigmoid', 'Hardswish', 'Hardtanh', 'HingeEmbeddingLoss', 'HuberLoss', 'Identity', 'InstanceNorm1d', 'InstanceNorm2d', 'InstanceNorm3d', 'KLDivLoss', 'L1Loss'

In [31]:
dftonp = test.values
nptonn = torch.from_numpy(dftonp)
nptonn.size(0)

672

In [78]:
nptonn.size()

torch.Size([672, 23])

In [29]:
device = torch.device("cuda:0")

In [102]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.input = 23
    self.hidden = 672
    self.output= 1
    self.model = nn.Sequential(
        nn.Linear(self.input, self.hidden), # input size, hidden size
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(self.hidden, int(self.hidden/2)),
        nn.ReLU(),
        nn.Linear(int(self.hidden/2),self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print(x.size())
    x = self.model(x)
    return x

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.input = 672
    self.output = 23
    self.hidden = 128
    self.model = nn.Sequential(
        nn.Linear(self.input, self.hidden), # input size, hidden size
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3),
        nn.Linear(self.hidden, int(self.hidden/2)),
        nn.LeakyReLU(0.2),
        nn.Linear(int(self.hidden/2),self.output), # hidden size, output size
        nn.Tanh()
    ).to(device)

  def forward(self, x):
    #print('generator',x.size())
    x = self.model(x)
    return x

# 모델 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)

g_optim = optim.Adam(generator.parameters(), lr=2e-4)
d_optim = optim.Adam(discriminator.parameters(), lr=2e-4)

criterion = nn.BCELoss().to(device)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=g_optim, mode='min', verbose=True, patience=10, factor=0.5)

import time
n_epochs = 10
sample_interval = 100 # 몇 번의 배치마다 결과를 출력할 것인가
noise = 672
start_time = time.time()
newdata = []
for epoch in range(n_epochs):
  dftonp = test.values
  nptonn = torch.from_numpy(dftonp).float()

  real = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(1.0) # 진짜
  fake = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(0.0) # 가짜

  real_data = nptonn.cuda()
  
  g_optim.zero_grad()
  z = torch.normal(mean=0, std=1, size=(nptonn.shape[0], noise)).cuda() # random noise
  #print('noise',z.size())
  generated_dis = generator(z) # create distribution
  generated_dis_value = generated_dis.detach().cpu()
  newdata.append(generated_dis)
  # criterion.cuda()
  g_loss =  criterion(discriminator(generated_dis), real) # calculate generator loss
  
  # update generator
  g_loss.backward()
  g_optim.step()

  # update discriminator
  real_loss = criterion(discriminator(real_data), real)
  fake_loss = criterion(discriminator(generated_dis.detach()), fake)
  d_loss = (real_loss + fake_loss) / 2

  d_loss.backward()
  d_optim.step()

  print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")


[Epoch 0/10] [D loss: 0.682218] [G loss: 0.676611] [Elapsed time: 0.01s]
[Epoch 1/10] [D loss: 0.476785] [G loss: 0.658415] [Elapsed time: 0.02s]
[Epoch 2/10] [D loss: 0.409846] [G loss: 0.641439] [Elapsed time: 0.03s]
[Epoch 3/10] [D loss: 0.394222] [G loss: 0.625326] [Elapsed time: 0.04s]
[Epoch 4/10] [D loss: 0.396720] [G loss: 0.608111] [Elapsed time: 0.05s]
[Epoch 5/10] [D loss: 0.404734] [G loss: 0.591713] [Elapsed time: 0.05s]
[Epoch 6/10] [D loss: 0.414251] [G loss: 0.574726] [Elapsed time: 0.06s]
[Epoch 7/10] [D loss: 0.425514] [G loss: 0.556766] [Elapsed time: 0.07s]
[Epoch 8/10] [D loss: 0.436643] [G loss: 0.540711] [Elapsed time: 0.08s]
[Epoch 9/10] [D loss: 0.449231] [G loss: 0.523282] [Elapsed time: 0.09s]


In [ ]:
newdata

In [105]:
nptonn[0]

tensor([0.0000, 0.0000, 3.2581, 4.4067, 6.2860, 0.6931, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.6931, 6.2860])